In [3]:
import os
import database
import json
from event_generator import event_generator

In [4]:
parsed_keys = {'globalGameId','teamId','playerId','relatedEventId','relatedPlayerId','field','endX',
               'endY','blockedX','blockedY','shot','goalMouthY','goalMouthZ','goal','ownGoal','isTouch':'touch',
               'id':'globalEventId','eventId':'localEventId','x':'eventX','y':'eventY'}

modified_keys ={'period','expandedMinute','minute','second','type','outcomeType','cardType'}

modified_quals = {44,59,212,213}
value_quals = {30:'involvedPlayers',56: 'zoneGeneral', 130:'teamFormation',131:'teamPlayerFormation',145:'formationSlot',194:'captainPlayerId'}
bool_quals = {1:'longBall',2:'cross',3:'headPass',4:'throughball',7:'playerCaughtOffside',8:'goalDisallowed',13:'foul',
              14:'lastMan',15:'head',29:'assisted',31:'yellowCard',32:'secondYellowCard',33:'redCard',74:'missHigh',
              82:'blocked',88:'highClaim',94:'outfielderBlock',100:'blockedClose',101:'savedOffLine',154:'intentionalAssist',
              155:'chipped',156:'layOff',169:'leadingToAttempt',170:'leadingToGoal',173:'parriedSafe',174:'parriedDanger',
              177:'collected',178:'standingSave',179:'divingSave',185:'blockedCross',186:'keeperMissed',
              187:'keeperSaved',188:'keeperWentWide',190:'fromShotOffTarget',210:'shotAssist',211:'overRun',214:'bigChance',
              241:'indirectFreekickTaken',242:'obstruction',264:'aerialFoul',285:'defensive',286:'offensive',11111:'intentionalGoalAssist',
              11112:'bigChanceCreated',11113:'keyPass',11114:'voidYellowCard',11115:'keeperSaveInTheBox',
              11116:'keeperSaveInSixYard',11117:'keeperSaveObox'}
lists = [({15,20,21,72,182,183},'bodyPart'),
         ({9,22,23,24,25,26,160},'shotSituation'),
         ({73,74,75,76,77,78,79,80,81},'direction'),
         ({16,17,18,19,60,61,62,63,64,65,66,67,68,69,70,71},'zone'),
         ({5,6,107,123,124},'passSituation')]

ignored_keys = {'qualifiers','satisfiedEventsTypes','$idx','$len','minuteInfo','satisfiers','text'}
ignored_quals = {28,55,102,103,140,141,146,147,212,213,233}

In [5]:
def parse_event(event):
    record = {}

    for k in parsed_keys:
        record[k] = event.get(k)
        
    record['period'] = event['period']['value']
    expandedMinute = event.get('expandedMinute')
    minute = event.get('minute')
    second = event.get('second')
    record['expandedMinute'] = expandedMinute if expandedMinute is not None and expandedMinute < 200 else None
    record['minute'] = minute if minute is not None and minute < 200 else None
    record['second'] = second if second is not None and second < 60 else None
    record['type'] = event['type']['value']
    
    outcome_type = event.get('outcomeType')
    if outcome_type:
        record['outcomeType'] = outcome_type.get('value')
    
    card_type = event.get('cardType')
    if card_type:
        record['cardType'] = card_type.get('value')
    
    for qual in event.get('qualifiers'):
        q = qual['type']['value']

        if q == 44: #playerPosition
            value = qual['value']
            if len(value) < 3:
                record['playerPosition'] = int(value)
            else:
                record['playerPositions'] = value
        
        elif q == 59: #jerseyNumber
            value = qual['value']
            if len(value) < 3:
                record['jerseyNumber'] = int(value)
            else:
                record['jerseyNumbers'] = value
        
        elif q in value_quals:
            record[value_quals[q]] = qual['value']
        
        elif q in bool_quals:
            record[bool_quals[q]] = True
            
        else:
            for lst in lists:
                if q in lst[0]:
                    record[lst[1]] = q
                    break
            else:
                if q not in ignored_quals:
                    print(f'New {qual = }')
                    raise AttributeError
    return record


def upload_events(count=None):
    records = []
    for n,event in enumerate(event_generator(count=count,restricted_types={'OffsideGiven'}),1):
        records.append(parse_event(event))
        if not n % (count // 2 if count else 50000):
            db = database.DataBase()
            for record in records:
                db.set_record('events',record,skip_existing=True)
            db.commit()
            db.close()
            records = []
    print('done')

In [1]:
def preprocess_record(record):   
    renamed_keys = {'isTouch':'touch','id':'globalEventId','eventId':'localEventId','x':'eventX','y':'eventY'}
    for k,v in renamed_keys.items():
        record[v] = record.get(k)

In [16]:
def parse_event_new(event):
    record = {}

    for k in parsed_keys:
        record[k] = event.get(k)
        
    for k,v in renamed_keys.items():
        record[v] = event.get(k)
        
    record['period'] = event['period']['value']
    expandedMinute = event.get('expandedMinute')
    minute = event.get('minute')
    second = event.get('second')
    record['expandedMinute'] = expandedMinute if expandedMinute is not None and expandedMinute < 200 else None
    record['minute'] = minute if minute is not None and minute < 200 else None
    record['second'] = second if second is not None and second < 60 else None
    record['type'] = event['type']['value']
    
    outcome_type = event.get('outcomeType')
    record['outcomeType'] = outcome_type.get('value') if outcome_type else None
    
    card_type = event.get('cardType')
    record['cardType'] = card_type.get('value') if card_type else None
    
    quals = {q['type']['value']:q.get('value') for q in event.get('qualifiers')}
    
    if 44 in quals: #playerPosition
        value = quals[44]
        if len(value) < 3:
            record['playerPosition'] = int(value)
            record['playerPositions'] = None
        else:
            record['playerPosition'] = None
            record['playerPositions'] = value
    else:
        record['playerPosition'] = None
        record['playerPositions'] = None
        
        
    if 59 in quals: #jerseyNumber
        value = quals[59]
        if len(value) < 3:
            record['jerseyNumber'] = int(value)
            record['jerseyNumbers'] = None
        else:
            record['jerseyNumber'] = None
            record['jerseyNumbers'] = value
    else:
        record['jerseyNumber'] = None
        record['jerseyNumbers'] = None
        
    
    for bool_qual in bool_quals:
        record[bool_quals[bool_qual]] = bool_qual in quals
    
    for value_qual in value_quals:
        record[value_quals[value_qual]] = quals[value_qual] if value_qual in quals else None

    for lst in lists:
        for q in lst[0]:
            if q in quals:
                record[lst[1]] = q
                break
        else:
            record[lst[1]] = None
    return record


def upload_events(count=None):
    records = []
    for n,event in enumerate(event_generator(count=count,restricted_types={'OffsideGiven'}),1):
        records.append(parse_event_new(event))
        if not n % (count // 2 if count else 50000):
            db = database.DataBase()
            db.set_record('events',records,many=True,skip_existing=True)
            db.commit()
            db.close()
            print('inserted batch')
            records = []
    print('done')

In [17]:
upload_events()

Event generator: started reading from events_1.json


OperationalError: 2055: Lost connection to MySQL server at 'localhost:3306', system error: 10053 Программа на вашем хост-компьютере разорвала установленное подключение